In [1]:
from os import path
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
METADATA_DESTINATION = '../dataset/imdb_crop/'
METADATA_CROPPED_FILE_MATLAB = METADATA_DESTINATION + 'imdb.mat'

In [2]:
'''
# RETRIEVING NAMES AND IMG URLS
names = open('names.html', 'r', errors='ignore')
actors = open("most_famous_actors.txt", 'w')

actors_names = []
img_urls = []
soup = BeautifulSoup(names, 'html.parser')
for span in soup.findAll('span'):
    for img in span.findAll('img'):
        if img:
            actors_names.append(img.attrs['alt'])
            img_urls.append(re.split(r' |,', img.attrs['srcset'])[-2])

for i in range(len(actors_names)):
    actors.write(actors_names[i] + ';' + img_urls[i] + '\n')
'''

'\n# RETRIEVING NAMES AND IMG URLS\nnames = open(\'names.html\', \'r\', errors=\'ignore\')\nactors = open("most_famous_actors.txt", \'w\')\n\nactors_names = []\nimg_urls = []\nsoup = BeautifulSoup(names, \'html.parser\')\nfor span in soup.findAll(\'span\'):\n    for img in span.findAll(\'img\'):\n        if img:\n            actors_names.append(img.attrs[\'alt\'])\n            img_urls.append(re.split(r\' |,\', img.attrs[\'srcset\'])[-2])\n\nfor i in range(len(actors_names)):\n    actors.write(actors_names[i] + \';\' + img_urls[i] + \'\n\')\n'

In [3]:
actors = open("most_famous_actors.txt", 'r')
actors = actors.read()
actors_names = re.split(r';|\n', actors)[::2]
actors_names = actors_names[:-1]
actors_img_urls = re.split(r';|\n', actors)[1::2]

In [ ]:
'''
# MOST FAMOUS ACTORS IN IMDB
head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'imdb.pickle')
df = pd.read_pickle(path_serialized)
df.head()

df_actors = pd.DataFrame(columns=['name', 'url_image', 'gender', 'age'])
with tqdm(total=len(actors_names)) as pbar:
        for i in range(len(actors_names)):
            q = 'name=="' + actors_names[i] + '"'
            query_results = df.query(q)[["age","gender"]]
            if not query_results.empty:
                new_row = query_results.iloc[0]
                new_row['name'] = actors_names[i]
                new_row['url_img'] = actors_img_urls[i]
                df_actors = df_actors.append(new_row)
                pbar.update(1)
df_actors.reset_index()

len(df_actors)


head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'imdb_most_famous_actors.pickle')
df_actors.to_pickle(path_serialized)

df_actors = pd.read_pickle(path_serialized)
df_actors.head()

df_actors.describe(include='all')
'''

In [80]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
# From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
sparql.setQuery("""
SELECT  DISTINCT ?gender (SAMPLE(?birth) as ?birth_date)
 (SAMPLE(?death_date) as ?dateOfDeath)  WHERE {
?id wdt:P31 wd:Q5.
?id wdt:P569 ?birth .
?id wdt:P21 ?gender.
OPTIONAL{?id wdt:P570 ?death_date .}
SERVICE wikibase:label {
bd:serviceParam wikibase:language "en".
?id rdfs:label ?idLabel .
}?id ?label "Natalie Portman"@en.
}
GROUP BY ?gender
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results_df = pd.io.json.json_normalize(results['results']['bindings'])

  results_df = pd.io.json.json_normalize(results['results']['bindings'])



In [81]:
# male Q6581097
# female Q6581072
print(results_df['gender.value'].values[0])
print(results_df['birth_date.value'].values[0])

http://www.wikidata.org/entity/Q6581072
1981-06-09T00:00:00Z
